In [ ]:
install.packages("tailDepFun")

In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

In [ ]:
# Code for simulations in Janssen & Wan (2019)
# Setting: d=20, k=2, model correctly specified

library(skmeans)
require(matrixStats)
require(gtools)
library(tailDepFun)
library(transport)
library(ramify)

In [ ]:
# below is the relevant content for the CRPS estimation from Yuen and Stoev, except that all "set.seed's" are deleted, since that interfered with the overall simulation
############# Begin of the CRPS part

## function maxlinear: same as A %*% Z except replaces plus
## operation by max
maxlinear <- function(A,Z){
  d <- nrow(A)
  p <- ncol(A)
  if(class(Z)=="numeric"){
    Z <- matrix(Z,length(Z),1)
  }
  if(p != nrow(Z)) stop("non-conformable arguments")
  n <- ncol(Z)
  X <- matrix(rowMaxs(A[rep(1:d,n),]*t(Z[,rep(1:n,each = d)])),d,n)
  return(X)
}
############# End of maxlinear

In [ ]:
################# Start of the k PC method ##########
#######################
#single iteration
#centroids is a k*d matrix with current proposals
clusterPC_iter=function(data, centroids){
  k=length(centroids[,1])
  n=length(data[,1])
  d=length(data[1,])
#     data-size(n*d); centroids-size(k*d);  so we transpose centroids
  M=data%*%t(centroids)
  #find current value, this value is our criteria corresponds to minimizing the expected c2 dissimilarity:
  v=mean(apply(M,1,max))
#     find the index gr where the maximum is obtained
  gr=argmax(M,rows=T)
  for (i in 1:k){
    seldata=data[gr==i,]
    if (length(seldata)==d) #interpretation problem when just one vector
      seldata=t(seldata)
#       sig is a d * d matrix of seldata^2
    Sig=t(seldata)%*%seldata/n
    res=eigen(Sig)
    centroids[i,]=abs(res$vectors[,1]) #use the first eigenvector, the entries of which are necessarily positive
#      the first eigenvector for each cluster as the new centroid_i
  }
  list(centroids,v,gr)
}
################
#pick randomly the initial centers
clusterPCOnce=function(data,k,tol,startFromMeans=FALSE){
  val=0
  n=length(data[,1])
  if (startFromMeans)
    centroids=clusterMeans(data,k)  
  else{
    centroids=data[sample(1:n,k),]
    if (k==1)   #make sure it is a matrix
      centroids=t(as.matrix(centroids))
  }
  niter=0
  repeat{
    niter=niter+1
    res=clusterPC_iter(data, centroids)
    centroids=res[[1]]
    diff=res[[2]]-val
    val=res[[2]]
    cluster=res[[3]]
    if(diff<tol)
      break
  }
#   print(niter)
  list(centroids,val,cluster)
}
###############
#iterate nrep times and pick the best
clusterPC=function(data,k,tol=10^(-5),nrep=100,startFromMeans=FALSE){
  maxval=0
  for( i in 1:nrep){
    res=clusterPCOnce(data,k,tol,startFromMeans && (i==1))
    if (res[[2]]>maxval){
      cluster=res[[3]]
      maxval=res[[2]]
      centroids=res[[1]]
    }
  }
#   centroids
list(centroids,maxval,cluster)
}

In [ ]:
############# Start of additional setup, necessary functions

# L2-norm is used below:
norm_vec <- function(x) sqrt(sum(x^2))


# Bmatrix transforms theta-parameters into a matrix A, makes sure that standardization is met
Bmatrix <- function(m,theta){
  B<- cbind(m,1-theta)
  return(B)
}



############# End of additional setup, necessary functions

In [ ]:
############ Start of core simulation part
m<-100
# m=number of simulations 
# Initialization if evaluation characteristics:
# sk relates to our method, spherical means
# dist* relates to d_s (Table 1), wasser* relates to W_1 (Tables 2 and 3)
wasserkPC2<-rep(0,m)
wassersk2<-rep(0,m)
duration_sk2<-rep(0,m)
duration_kPC2<-rep(0,m)

wasserkPC4<-rep(0,m)
wassersk4<-rep(0,m)
duration_sk4<-rep(0,m)
duration_kPC4<-rep(0,m)

set.seed(10)


In [ ]:
# misspecified to k = 2

for(i in 1:m){
print(i)
# Random model initialization, see paper for model description
theta1 <- c(runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,
           runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,
           runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,
           runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,runif(1)*0.5)
theta2 <- c(runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,
           runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,runif(1)*0.5,
           0,0,0,0,0,0,0,0,0,0)
temp_sum <- theta1 + theta2 
first_m <- cbind(theta1, theta2)
B <- Bmatrix(first_m,temp_sum)

# C are the true points of support (as rows) of the spectral measure and trueprobs the corresponding probabilities
C<-t(cbind(B[,1]/norm_vec(B[,1]),B[,2]/norm_vec(B[,2]),B[,3]/norm_vec(B[,3])))
trueprobs <- apply(B, 2, norm_vec)/sum(apply(B, 2, norm_vec))


n<-1000
# n=sample size per simulation
# Generate Z vectors and sample data
Z <- matrix(1/rexp(3*n),3,n)
data <- t(maxlinear(B,Z))
# Transform to unit Pareto marginals via rank transformation for sk-means procedure 
x <- apply(data, 2, function(i) n/(n + 1 - rank(i)))

# For k-means estimator, chose transformed observations with largest L2-norms
l2norm<-apply(x, 1, norm_vec)
# we use the largest 100 observations:
u<-0.90
ex<-x[l2norm>quantile(l2norm,u),]/l2norm[l2norm>quantile(l2norm,u)]

# apply spherical 2-means:
start_time_sk <- Sys.time()
skout<-skmeans(ex,2,control=list(nruns = 50))
end_time_sk <- Sys.time()
# estimated points of mass (as rows) and corresponding probabilities
Csk<-skout$prototypes
skprobs<-table(skout$cluster)/sum(table(skout$cluster))

# The characteristic W_1 is derived below, with the help of the package 'transport'
wassersk2[i]<-wasserstein(wpp(C,trueprobs),wpp(Csk,skprobs),p=1)
           
# record running time for each specification
duration_sk2[i]<- round(end_time_sk - start_time_sk,4)
print("k-means misspecification k = 2")
print(c(wassersk2[i],duration_sk2[i]))

# run the current simulated data with spherical k-PC method          
# apply kPC 2-means:
start_time_pc <- Sys.time()
kPCout<-clusterPC(ex,2)
end_time_pc <- Sys.time()
# estimated points of mass (as rows) and corresponding probabilities
CkPC<-kPCout[[1]]
kPCprobs<-table(kPCout[[3]])/sum(table(kPCout[[3]])) 

# The characteristic W_1 is derived below, with the help of the package 'transport'
wasserkPC2[i]<-wasserstein(wpp(C,trueprobs),wpp(CkPC,kPCprobs),p=1)

#record the running time of kPC
duration_kPC2[i]<- round(end_time_pc - start_time_pc,4)

print("k-Principle Component misspecification k = 2")
print(c(wasserkPC2[i],duration_kPC2[i]))
           
# apply spherical 4-means:
start_time_sk <- Sys.time()
skout<-skmeans(ex,4,control=list(nruns = 100))
end_time_sk <- Sys.time()
# estimated points of mass (as rows) and corresponding probabilities
Csk<-skout$prototypes
skprobs<-table(skout$cluster)/sum(table(skout$cluster))

# The characteristic W_1 is derived below, with the help of the package 'transport'
wassersk4[i]<-wasserstein(wpp(C,trueprobs),wpp(Csk,skprobs),p=1)
           
# record running time for each specification
duration_sk4[i]<- round(end_time_sk - start_time_sk,4)
print("k-means misspecification k = 4")
print(c(wassersk4[i],duration_sk4[i]))

# run the current simulated data with spherical k-PC method          
# apply kPC 2-means:
start_time_pc <- Sys.time()
kPCout<-clusterPC(ex,4)
end_time_pc <- Sys.time()
# estimated points of mass (as rows) and corresponding probabilities
CkPC<-kPCout[[1]]
kPCprobs<-table(kPCout[[3]])/sum(table(kPCout[[3]])) 

# The characteristic W_1 is derived below, with the help of the package 'transport'
wasserkPC4[i]<-wasserstein(wpp(C,trueprobs),wpp(CkPC,kPCprobs),p=1)

#record the running time of kPC
duration_kPC4[i]<- round(end_time_pc - start_time_pc,4)

print("k-Principle Component misspecification k = 4")
print(c(wasserkPC4[i],duration_kPC4[i]))
}

In [ ]:
round(mean(wassersk2),4)
round(sd(wassersk2),4)
round(mean(wassersk4),4)
round(sd(wassersk4),4)


In [ ]:
round(mean(wasserkPC2),4)
round(sd(wasserkPC2),4)
round(mean(wasserkPC4),4)
round(sd(wasserkPC4),4)

In [ ]:
round(mean(duration_sk2),4)
round(sd(duration_sk2),4)
round(mean(duration_sk4),4)
round(sd(duration_sk4),4)

In [ ]:
round(mean(duration_kPC2),4)
round(sd(duration_kPC2),4)
round(mean(duration_kPC4),4)
round(sd(duration_kPC4),4)